In [28]:
def newton(f, df, x0, max_iter=100):
    """
    Newton's method for finding a root of f(x) = 0.
    f: function whose root is sought.
    df: derivative of f.
    x0: initial guess.
    tol: tolerance for stopping.
    max_iter: maximum number of iterations.
    Returns an approximate root.
    """
    x = x0
    for i in range(max_iter):
        fx = f(x)
        dfx = df(x)
        if dfx == 0:
            raise ValueError("Zero derivative encountered; try a different initial guess.")
        x_new = x - fx/dfx
        if abs(x_new - x) < 0.00001:
            return x_new
        x = x_new
    return x

# (a) f(x) = x^3 - 2x^2 - 5
def f_a(x):
    return x**3 - 2*x**2 - 5

def df_a(x):
    return 3*x**2 - 4*x

# (b) f(x) = x^3 + 3x^2 - 1
def f_b(x):
    return x**3 + 3*x**2 - 1

def df_b(x):
    return 3*x**2 + 6*x

# (c) f(x) = x^3 - x - 1
def f_c(x):
    return x**3 - x - 1

def df_c(x):
    return 3*x**2 - 1

# (d) f(x) = x^4 + 2x^2 - x - 3
def f_d(x):
    return x**4 + 2*x**2 - x - 3

def df_d(x):
    return 4*x**3 + 4*x - 1

# Find the roots with appropriate initial guesses.

# (a) There is one real root between 2 and 3.
root_a = newton(f_a, df_a, x0=2.5)
print("(a) Root:", root_a, "f(root) =", f_a(root_a))

# (b) f(x)= x^3 + 3x^2 - 1 has three real roots.
# We'll choose three initial guesses.
root_b1 = newton(f_b, df_b, x0=0.5)   # Expecting a root in (0,1)
root_b2 = newton(f_b, df_b, x0=-0.5)  # Expecting a root in (-1,0)
root_b3 = newton(f_b, df_b, x0=-3)    # Expecting a root in (-3,-2)
print("(b) Roots:")
print("    Root 1:", root_b1, "f(root1) =", f_b(root_b1))
print("    Root 2:", root_b2, "f(root2) =", f_b(root_b2))
print("    Root 3:", root_b3, "f(root3) =", f_b(root_b3))

# (c) f(x)= x^3 - x - 1 has one real root between 1 and 2.
root_c = newton(f_c, df_c, x0=1.5)
print("(c) Root:", root_c, "f(root) =", f_c(root_c))

# (d) f(x)= x^4 + 2x^2 - x - 3 has two real roots.
# We'll choose one guess in (-1,0) and one in (1,2).
root_d1 = newton(f_d, df_d, x0=-0.5)
root_d2 = newton(f_d, df_d, x0=1.5)
print("(d) Roots:")
print("    Root 1:", root_d1, "f(root1) =", f_d(root_d1))
print("    Root 2:", root_d2, "f(root2) =", f_d(root_d2))


(a) Root: 2.6906474480286153 f(root) = 1.7763568394002505e-14
(b) Roots:
    Root 1: 0.5320888862414667 f(root1) = 1.41895384331292e-11
    Root 2: -0.6527036446661393 f(root2) = -1.1102230246251565e-16
    Root 3: -2.8793852415718164 f(root3) = 3.552713678800501e-15
(c) Root: 1.3247179572447898 f(root) = 1.865174681370263e-13
(d) Roots:
    Root 1: -0.8760531158171996 f(root1) = 6.146194664324867e-13
    Root 2: 1.1241230297043154 f(root2) = 0.0


In [2]:
import numpy as np

def newton_method(f, df, x0, tol=1e-5, max_iter=100):
    """Finds a real root of f using Newton's method, ensuring uniqueness."""
    x = x0
    for _ in range(max_iter):
        fx = f(x)
        dfx = df(x)
        if abs(fx) < tol:
            return round(x, 5)  # Return root with rounding to avoid duplicate issues
        x -= fx / dfx  # Newton's update step
    return None  # No convergence


def polynomial_div(f, root):
    """Reduces the polynomial degree by dividing it by (x - root) using NumPy's polydiv()."""
    quotient, remainder = np.polydiv(f.coeffs, [1, -root])
    
    # Ensure remainder is approximately zero
    if not np.allclose(remainder, 0, atol=1e-5):
        print(f"Warning: Root {root} may not be exact, remainder: {remainder}")

    return np.poly1d(quotient)  # Return the deflated polynomial

def muller(f, p0, p1, p2, tol=1e-5, max_iter=100):
    """Finds a root of f using Muller's method, which handles complex roots."""
    p0, p1, p2 = complex(p0), complex(p1), complex(p2)

    for _ in range(max_iter):
        h1 = p1 - p0
        h2 = p2 - p1
        delta1 = (f(p1) - f(p0)) / h1
        delta2 = (f(p2) - f(p1)) / h2
        d = (delta2 - delta1) / (h2 + h1)

        b = delta2 + h2 * d
        disc = b**2 - 4 * f(p2) * d  # Discriminant
        D = np.sqrt(complex(disc))  # Fix: Use `complex()` instead of `np.complex`

        # Choose the denominator that avoids cancellation
        E = b + D if abs(b - D) < abs(b + D) else b - D
        h = -2 * f(p2) / E
        p = p2 + h  # Next approximation

        if abs(h) < tol:
            return p  # Converged to root

        # Shift points for next iteration
        p0, p1, p2 = p1, p2, p

    return p  # Return last computed value if no exact convergence

def find_all_roots(f, df, initial_guesses):
    """Finds all real and complex roots of a polynomial, ensuring uniqueness."""
    real_roots = set()  # Using a set to store unique real roots
    complex_roots = []

    # Step 1: Find real roots using Newton's method
    for x0 in initial_guesses:
        root = newton_method(f, df, x0)
        if root is not None and root not in real_roots:
            real_roots.add(root)
            f = polynomial_div(f, root)  # Reduce polynomial degree

    # Step 2: Convert real_roots set to a sorted list
    real_roots = sorted(real_roots)

    # Step 3: Find complex roots using Muller's method
    remaining_roots = []
    x0, x1, x2 = complex(0), complex(1), complex(-1)  # Generic initial guesses
    
    while f.order > 1:  # Muller's method works best on quadratics
        root = muller(f, x0, x1, x2)
        remaining_roots.append(root)
        f = polynomial_div(f, root)

    return real_roots + remaining_roots  # Combine real and complex roots


# Define polynomials and their derivatives
polynomials = [
    np.poly1d([1, 5, -9, -85, -136]),     # f(x) = x^4 + 5x^3 - 9x^2 - 85x - 136
    np.poly1d([1, -2, -12, 16, -40]),     # f(x) = x^4 - 2x^3 - 12x^2 + 16x - 40
    np.poly1d([1, 1, 3, 2, 2]),           # f(x) = x^4 + x^3 + 3x^2 + 2x + 2
    np.poly1d([1, 11, -21, -10, -21, -5]) # f(x) = x^5 + 11x^4 - 21x^3 - 10x^2 - 21x - 5
]

derivatives = [p.deriv() for p in polynomials]  # Compute derivatives for Newton's method

# Initial guesses for Newton’s method
initial_guesses_list = [
    [-4, -1, 2, 4],   # Guesses for polynomial 1
    [-3, 0, 3, 5],    # Guesses for polynomial 2
    [-2, 0, 2, 4],    # Guesses for polynomial 3
    [-4, -1, 1, 3, 5] # Guesses for polynomial 4
]

# Run the tests and find all roots
for i, (f, df, initial_guesses) in enumerate(zip(polynomials, derivatives, initial_guesses_list), start=1):
    roots = find_all_roots(f, df, initial_guesses)
    print(f"Roots of polynomial {i}: {roots}")


Roots of polynomial 1: [-4.12311, (-2.499998023038043-1.3228793916445514j), (-2.4999980230379193+1.3228793916445762j)]
Roots of polynomial 2: [-3.54823, (0.5835599590826792+1.4941797304444835j), (0.583559959082679-1.4941797304444833j)]
Roots of polynomial 3: [(-0.500000000067848+0.8660254035239285j), (-0.4999999997404676-0.8660254037129417j), (-1.6266845687489017e-10-1.4142135623463592j)]
Roots of polynomial 4: [-0.25024, 2.26009, (-0.19871014719556765+0.8133207822937638j), (-0.19871014719556765-0.8133207822937638j)]


In [3]:
import numpy as np

def newton_method(f, df, x0, tol=1e-5, max_iter=100):
    """
    Finds a real root of f using Newton's method.
    Only returns a root if the iterates remain real (or have negligible imaginary parts).
    """
    x = x0  # initial guess should be a real number
    for _ in range(max_iter):
        fx = f(x)
        dfx = df(x)
        if abs(fx) < tol:
            # Convergence achieved: check for small imaginary component
            if isinstance(x, complex) and abs(x.imag) >= tol:
                return None  # Not a valid real root
            return round(x.real, 5)  # Return the real part rounded
        if abs(dfx) < tol:
            return None  # Avoid division by nearly zero
        x_new = x - fx / dfx
        # Ensure that the new iterate remains real:
        if isinstance(x_new, complex) and abs(x_new.imag) >= tol:
            return None  # If the imaginary part is significant, abort
        x = x_new.real if isinstance(x_new, complex) else x_new
    return None  # Did not converge within max_iter

def polynomial_div(f, root, tol=1e-5, round_decimals=8):
    """
    Deflates polynomial f by dividing it by (x - root) with improved rounding.
    """
    quotient, remainder = np.polydiv(f.coeffs, [1, -root])
    if not np.allclose(remainder, 0, atol=tol):
        # You can remove the warning if desired
        # print(f"Warning: Root {root} may not be exact, remainder: {remainder}")
        pass
    quotient = np.round(quotient, round_decimals)
    return np.poly1d(quotient)

def muller(f, p0, p1, p2, tol=1e-5, max_iter=100):
    """
    Finds a root of f using Muller's method (which works in the complex domain).
    This is used to find the remaining complex roots after extracting the real roots.
    """
    p0, p1, p2 = complex(p0), complex(p1), complex(p2)
    for _ in range(max_iter):
        h1 = p1 - p0
        h2 = p2 - p1
        delta1 = (f(p1) - f(p0)) / h1
        delta2 = (f(p2) - f(p1)) / h2
        d = (delta2 - delta1) / (h2 + h1)
        b = delta2 + h2 * d
        disc = b**2 - 4 * f(p2) * d
        D = np.sqrt(complex(disc))
        E = b + D if abs(b - D) < abs(b + D) else b - D
        h = -2 * f(p2) / E
        p = p2 + h
        if abs(h) < tol:
            return p  # Convergence achieved
        p0, p1, p2 = p1, p2, p
    return p

def find_all_roots(f, df, initial_guesses, tol=1e-5):
    """
    Finds all real and complex roots of a polynomial.
    Uses Newton's method (restricted to real roots) first, then Muller's method for complex roots.
    """
    total_roots = f.order  # Total number of roots expected
    real_roots = set()      # To store unique real roots
    complex_roots = []      # To store complex roots

    # Step 1: Find real roots using Newton's method
    for x0 in initial_guesses:
        if len(real_roots) >= total_roots:
            break
        root = newton_method(f, df, x0, tol=tol)
        if root is not None and root not in real_roots:
            real_roots.add(root)
            f = polynomial_div(f, root, tol=tol)  # Deflate the polynomial

    real_roots = sorted(real_roots)

    # Step 2: Use Muller's method for the remaining (complex) roots
    x0, x1, x2 = 0, 1, -1  # Generic initial guesses for Muller's method
    while f.order > 0 and (len(real_roots) + len(complex_roots)) < total_roots:
        root = muller(f, x0, x1, x2, tol=tol)
        # If the imaginary part is negligible, treat as real
        if abs(root.imag) < tol:
            root = round(root.real, 5)
        # To handle conjugate pairs (for polynomials with real coefficients)
        if isinstance(root, complex) and abs(root.imag) >= tol:
            conjugate_root = np.conj(root)
            complex_roots.extend([root, conjugate_root])
            f = polynomial_div(f, root, tol=tol)
            f = polynomial_div(f, conjugate_root, tol=tol)
        else:
            real_roots.append(root)
            f = polynomial_div(f, root, tol=tol)
    return real_roots + complex_roots

# Example polynomials and derivatives
polynomials = [
    np.poly1d([1, 5, -9, -85, -136]),     # f(x) = x^4 + 5x^3 - 9x^2 - 85x - 136
    np.poly1d([1, -2, -12, 16, -40]),       # f(x) = x^4 - 2x^3 - 12x^2 + 16x - 40
    np.poly1d([1, 1, 3, 2, 2]),             # f(x) = x^4 + x^3 + 3x^2 + 2x + 2
    np.poly1d([1, 11, -21, -10, -21, -5])    # f(x) = x^5 + 11x^4 - 21x^3 - 10x^2 - 21x - 5
]
derivatives = [p.deriv() for p in polynomials]

initial_guesses_list = [
    [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
    [-6, -4, -2, 0, 2, 4, 6],
    [-5, -3, -1, 1, 3, 5],
    [-6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6]
]

for i, (f, df, initial_guesses) in enumerate(zip(polynomials, derivatives, initial_guesses_list), start=1):
    roots = find_all_roots(f, df, initial_guesses)
    print(f"Roots of polynomial {i}: {roots}")



Roots of polynomial 1: [-4.12311, 4.12311, (-2.4999980231468033-1.3228793913953443j), (-2.4999980231468033+1.3228793913953443j)]
Roots of polynomial 2: [-3.54823, 4.38111, (0.58356+1.4941796265509713j), (0.58356-1.4941796265509713j)]
Roots of polynomial 3: [(-0.500000000067848+0.8660254035239285j), (-0.500000000067848-0.8660254035239285j), 1.4142135623730951j, -1.4142135623730951j]
Roots of polynomial 4: [-0.25024, 2.26009, -12.61243, (-0.19871014740731383+0.8133207826889866j), (-0.19871014740731383-0.8133207826889866j)]
